In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/2022-2023 Awards Project clean/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 2 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

31 October 2022 ; 11:07:31


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,Precious Achiuwa,C,23,TOR,6,0,23.2,3.0,8.5,0.353,...,-1.33,110.95,112.39,-1.44,0.2,111.25,110.99,0.26,0,1
2,Dalano Banton,PG,23,TOR,6,0,8.2,1.0,2.5,0.400,...,-1.33,110.95,112.39,-1.44,0.2,111.25,110.99,0.26,0,1
4,Khem Birch,C,30,TOR,2,0,6.5,0.5,1.0,0.500,...,-1.33,110.95,112.39,-1.44,0.2,111.25,110.99,0.26,0,1
5,Chris Boucher,PF,30,TOR,3,0,17.0,4.0,6.7,0.600,...,-1.33,110.95,112.39,-1.44,0.2,111.25,110.99,0.26,0,1
6,Justin Champagnie,SF,21,TOR,1,0,3.0,1.0,1.0,1.000,...,-1.33,110.95,112.39,-1.44,0.2,111.25,110.99,0.26,0,1


In [5]:
# Set training data
train_data = train_data[train_data['Year'] < 2020] # DELETE THIS LINE
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Trey Murphy III,31.7,6,3,NOP,16.8,2.733333,9.566667,15.0,5,2.7,6.833333,0.380255
1,Dean Wade,24.8,6,3,CLE,17.1,4.100000,10.933333,10.5,2,4.7,6.833333,0.285181
2,Bobby Portis,24.2,5,0,MIL,20.5,1.640000,8.200000,13.0,1,1.8,3.280000,0.241782
3,Kevin Love,20.0,6,0,CLE,20.6,4.100000,8.200000,11.7,2,5.9,4.100000,0.218551
4,Nick Richards,21.2,6,0,CHO,26.0,2.733333,10.933333,12.5,5,4.8,9.566667,0.215246


In [7]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)